   Создание кластеров по баллам ЕГЭ и по баллам за 1 семестр: ищем 20 ближайших соседей там и там. Далее, проводим сравнение кластеров, то есть для каждого объекта смотрим как изменилось количество его ближайших соседей.
Подготовка: выводим информацию по каждой специальности 

In [3]:
#Функция для удаления дубликатов
def unique(lst):
    seen = set()
    result = []
    for x in lst:
        if x in seen:
            continue
        seen.add(x)
        result.append(x)
    return result


#Функция перемножения множеств
def inters(a):
    s=set(a[0])
    for lst in a:
        s=s.intersection(set(lst))
    return(s)   

In [4]:
#Ищем id студентов выбранной специальности
#вывести список групп
def spec():
    import sqlite3
    #Создание списка из всех направлений...
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT grp_spec_grp_spec_name FROM stud_spisok "
    cursor.execute(sql)
    #print(cursor.fetchall())
    specials=cursor.fetchall()

    conn.commit()
    conn.close()

    #Делаем список из специальностей
    specs=[]
    for i in range(len(unique(specials))):
        specs.append(unique(specials)[i][0])
    return(specs)

In [7]:
#Ищем id студентов выбранной специальности
def kods(specs):
    import sqlite3
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT stud_kod FROM stud_spisok WHERE grp_spec_grp_spec_name = ?"
    cursor.execute(sql,[(specs)])
    #print(cursor.fetchall())
    students_kod=cursor.fetchall()

    conn.commit()
    conn.close()

    kod=[]
    for i in range(len(students_kod)):
        kod.append(students_kod[i][0])
    return(kod)

In [8]:
kod=kods(spec()[0])
#kod

In [13]:
#Вывод баллов ЕГЭ для выбранного студента:
def identyEge(IDENTY, Ege_predmet):
    import sqlite3
 
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT ball FROM raiting_ege WHERE stud_kod = ? AND predmet_name=?"
    cursor.execute(sql,[(IDENTY),(Ege_predmet)])
    #print(cursor.fetchall())
    student_ege=cursor.fetchall()
    conn.commit()
    conn.close()
    if student_ege==[]:return(0)
    else: return(student_ege[0][0])

In [14]:
identyEge(kod[0], 'Русский язык')

76

In [15]:
#ВЫбираем 3 балла ЕГэ по каждому студенту:
ege_spis=[]
for i in range(len(kod)):
    ege_spis.append([kod[i],identyEge(kod[i], 'Русский язык'),identyEge(kod[i], 'Математика'),identyEge(kod[0], 'Обществознание')])

In [17]:
#Эвклидово расстояние
def EuclideanDistance(list1, list2):
    
    #Если списки не равны, то уравниваем их:
    if len(list1)!=len(list2):
        if len(list1)>len(list2):
            list2+=[0]*abs(len(list1)-len(list2))
        else: 
            list1+=[0]*abs(len(list1)-len(list2))
                           
    Eukl=0
    for i in range(len(list1)):
        Eukl+=(list1[i]-list2[i])**2
    return(Eukl**(1/2))

In [18]:
#Строим список к(20)-ближайших соседей для каждого студента
import numpy as np
itog_list=[]

for j in range(len(kod)):
    list_of_neibh=[]
    for i in range(len(ege_spis)):
        list_of_neibh.append([ege_spis[i][0],EuclideanDistance(ege_spis[j][1:], ege_spis[i][1:])])
    list_of_neibh.sort(key=lambda x: x[1])
    itog_list.append([ege_spis[j][0],list(np.array(list_of_neibh[1:21])[:,0])])

In [115]:
kod[:3]

[50095, 50104, 50137]

In [9]:
#ege_spis

In [19]:
itog_list.sort()
#itog_list

Теперь сделаем тоже самое для баллов за 1 семестр:

In [119]:


#Выводим оценки за выбранный предмет для каждого id:
#Ищем id студентов выбранной специальности
def markses(ID, predmet,semak):
    import sqlite3
 
    
    conn = sqlite3.connect("stud_spisok_ege.db")
    cursor = conn.cursor()
    
    #matan='Математический анализ'

    #выбираем прошедшие данные чтобы их вывести...
    sql = "SELECT ball FROM stud_spisok_ball WHERE stud_kod = ? AND predmet_name=? AND semestr=?"
    cursor.execute(sql,[(ID),(predmet),(semak)])
    #print(cursor.fetchall())
    marks=cursor.fetchall()
    
    
    sql_11 = "SELECT ball FROM stud_spisok_ball WHERE predmet_name=?"
    cursor.execute(sql_11,[(predmet)])
    maximum=cursor.fetchall()
    
    conn.commit()
    conn.close()
    
    new_11=0
    for i in maximum:
        if i[0]==None: continue
        if new_11<i[0]: new_11=i[0]

    new=[]
    for i in marks:
        if i[0]==None: new.append(0)
        else:new.append(i[0]*100/new_11)
    return(new) 

In [123]:
markses(kod[0],'Математический анализ',1)

[50.0, 87.5, 37.5]

In [199]:
marks_list=[]
for i in range(len(kod)):
    if(len(markses(kod[i],'Линейная алгебра',1))>0 and len(markses(kod[i],'Философия',1))>0 and len(markses(kod[i],'Математический анализ',1))>0 and len(markses(kod[129],'Психология',1))>0):
        marks_list.append([kod[i],markses(kod[i],'Философия',1)[-1],markses(kod[i],'Математический анализ',1)[-1],markses(kod[i],'Линейная алгебра',1)[-1],markses(kod[i],'Психология',1)[-1]])

In [20]:
#marks_list

In [206]:
#Строим список к(20)-ближайших соседей для каждого студента но уже по экзаменам
import numpy as np
itog_list_b=[]

for j in range(len(marks_list)):
    list_of_neibh_b=[]
    for i in range(len(marks_list)):
        list_of_neibh_b.append([marks_list[i][0],EuclideanDistance(marks_list[j][1:], marks_list[i][1:])])
    list_of_neibh_b.sort(key=lambda x: x[1])
    itog_list_b.append([marks_list[j][0],list(np.array(list_of_neibh_b[1:21])[:,0])])

In [21]:
#itog_list_b.sort()
#itog_list_b

In [228]:
mass=[]
for i in range(len(itog_list_b)):
    sch=0
    for j in range(len(itog_list)):
        if itog_list_b[i][0]==itog_list[j][0]:
            for k in range(len(itog_list[j])):
                if itog_list[j][k] in itog_list_b[i]:
                    sch+=1
            break
    mass.append(sch)

In [1]:
#mass

In [220]:
s=0
for i in range(len(itog_list_b)):
    sch=0
    for j in range(len(itog_list_b[i])):
        if itog_list_b[i][0]==itog_list[j][0]:
            sch+=1
            break

In [221]:
s

129